In [1]:
import requests
import bs4
from lxml import etree
import pandas as pd
import re, json
from user_agent import generate_user_agent

In [2]:
with open("./links.json") as f:
    pages = json.load(f)

In [3]:
def get_reviews(link):
    results = []
    headers = {
        'User-Agent': generate_user_agent(device_type="desktop", os=('mac', 'linux'))
    }
    req = requests.get(link)
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    review_sel = "#w-opinions-widget > div.opinion-container div.opinion-item"
    rating_sel = "div.opinion-rating > div"
    adv_sel = "ul.descriptions-items > li:nth-child(1) > span:nth-child(2)"
    disadv_sel = "ul.descriptions-items > li:nth-child(2) > span:nth-child(2)"
    comment_sel = "ul.descriptions-items > li:nth-child(3) > span:nth-child(2)"

    review_list = parser.select(review_sel)
    
    for review_elm in review_list:
        rating = review_elm.select_one(rating_sel)
        advantages = review_elm.select_one(adv_sel)
        disadvantages = review_elm.select_one(disadv_sel)
        comment = review_elm.select_one(comment_sel)
        results.append(
            {
                "rating": rating["data-rating"] if rating else None,
                "advantages": advantages.get_text() if advantages else None,
                "disadvantages": disadvantages.get_text() if disadvantages else None,
                "comment": comment.get_text() if comment else None,
                "full_text": 
                    (advantages.get_text() if advantages else "") +
                    (disadvantages.get_text() if disadvantages else "") +
                    (comment.get_text() if comment else "")
            }
        )
    return results

In [4]:
results = [
    result for page_link in pages for result in get_reviews(page_link)
]

In [63]:
with open("./reviews01.json", "w") as f:
    json.dump(results, f)